In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from functions import merge_data

## load merged data

In [2]:
ahrf_data = 'data/hrsa/data_AHRF_2018-2019/processed/df_renamed.pkl'
usafacts_data_cases = 'data/usafacts/confirmed_cases_mar22.csv'
diabetes = 'data/diabetes/DiabetesAtlasCountyData.csv'
voting = 'data/voting/county_voting_processed.pkl'
df = merge_data.merge_data(ahrf_data=ahrf_data, 
                           usafacts_data_cases=usafacts_data_cases,
                           medicare_group="All Beneficiaries",
                           voting=voting,
                           diabetes=diabetes) # also cleans usafacts data

/accounts/projects/vision/chandan/covid-19-geographic-risk-prediction/functions/merge_data.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  diabetes = pd.read_csv(diabetes, skiprows = 2, skipfooter = 1)


In [3]:
df.keys()

Index(['id', 'Header-FIPSStandCtyCode', 'EntityofFile',
       'SecondaryEntityOfFile', 'DateofFile', 'DateofCreation', 'FileLength',
       'StateName', 'StateNameAbbreviation', 'CountyName',
       ...
       'condition_Heart Failure', 'condition_Hepatitis',
       'condition_Hyperlipidemia', 'condition_Hypertension',
       'condition_Ischemic Heart Disease', 'condition_Osteoporosis',
       'condition_Psychotic Disorders', 'condition_Stroke',
       'Diabetes Percentage', 'dem_to_rep_ratio'],
      dtype='object', length=7234)

## load data separately

In [4]:
from os.path import join as oj
from functions import load_medicare_data
ahrf_data = 'data/hrsa/data_AHRF_2018-2019/processed/df_renamed.pkl'
usafacts_data_cases = 'data/usafacts/confirmed_cases_mar22.csv'
usafacts_data_deaths = 'data/usafacts/deaths_mar22.csv'
diabetes_data = 'data/diabetes/DiabetesAtlasCountyData.csv'

# load data
facts = pd.read_pickle(ahrf_data)
facts = facts.rename(columns={'Blank': 'id'})

cases = pd.read_csv(usafacts_data_cases, encoding="iso-8859-1")
cases = cases.rename(columns={k: '#Cases_' + k for k in cases.keys() 
                              if not 'county' in k.lower()
                              and not 'state' in k.lower()})

chronic_all_orig = load_medicare_data.loadChronicSheet("All Beneficiaries")

diabetes = pd.read_csv(diabetes_data, skiprows = 2, skipfooter = 1)
diabetes = diabetes[["CountyFIPS", "Percentage"]]
diabetes.columns = ["countyFIPS", "Diabetes Percentage"]

/Users/Tiffany/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


In [5]:
#visualize the top 5 rows
cases.head(5)
cases.dtypes

countyFIPS           int64
County Name         object
State               object
stateFIPS            int64
#Cases_1/22/2020     int64
                     ...  
#Cases_3/17/2020     int64
#Cases_3/18/2020     int64
#Cases_3/19/2020     int64
#Cases_3/20/2020     int64
#Cases_3/21/2020     int64
Length: 64, dtype: object

In [6]:
chronic_all_orig.head(5)

,State,County,countyFIPS,condition_Alcohol Abuse,condition_Alzheimers,condition_Arthritis,condition_Asthma,condition_Atrial Fibrillation,condition_Autism,condition_Cancer,...,condition_Drug Abuse,condition_HIV/AIDS,condition_Heart Failure,condition_Hepatitis,condition_Hyperlipidemia,condition_Hypertension,condition_Ischemic Heart Disease,condition_Osteoporosis,condition_Psychotic Disorders,condition_Stroke
2,Alabama,Autauga,1001.0,2.4273,10.4989,35.0029,4.5463,8.5918,NaN,8.4762,...,3.2171,0.2504,15.8544,0.5201,48.9694,63.7064,31.3234,6.4149,3.0437,4.0647
3,Alabama,Baldwin,1003.0,2.1230,10.9441,37.6864,4.8890,9.4952,0.1046,8.5693,...,3.9631,0.1627,12.8695,0.5114,44.0747,60.3339,31.8638,6.2682,2.0997,3.8740
4,Alabama,Barbour,1005.0,4.7292,11.4923,37.5398,5.5134,6.8856,NaN,8.3313,...,4.9743,0.6616,13.7711,0.4166,46.5572,67.2384,26.4151,4.7047,4.5087,4.5822
5,Alabama,Bibb,1007.0,2.8803,10.8011,37.7588,4.9055,8.9559,0.0000,7.1557,...,4.4554,NaN,17.3717,0.9001,48.5149,67.9118,28.0378,6.1206,3.6454,5.5356
6,Alabama,Blount,1009.0,2.0132,11.4934,34.2423,5.3258,8.4004,NaN,6.7350,...,4.8499,NaN,16.2518,0.6772,46.4861,63.7811,30.5820,5.7284,2.8184,4.4290


In [7]:
diabetes.head(5)

,countyFIPS,Diabetes Percentage
0,1001,9.9
1,1003,8.5
2,1005,15.7
3,1007,13.3
4,1009,14.9


# clean cases

In [8]:
# remove cases whose county is unknown
cases = cases[cases.countyFIPS != 0]

# raw.iloc[224, 0] = 13245 # fix err with Richmond, Georgia

# sum over duplicate counties
# cases = cases.groupby(['countyFIPS', 'County Name', 'State', 'stateFIPS']).sum().reset_index()
cases = cases.groupby(['countyFIPS']).sum().reset_index()

# add id
# cases['id'] = cases.index

# merge data

In [12]:
facts['countyFIPS'] = facts['Header-FIPSStandCtyCode'].astype(int)
chronic_all_orig['countyFIPS'] = chronic_all_orig['countyFIPS'].astype(int)
df = pd.merge(facts, cases, on='countyFIPS')
df = pd.merge(df, chronic_all_orig, on='countyFIPS')
df = pd.merge(df, diabetes, on='countyFIPS')

In [13]:
df.keys()

Index(['id', 'Header-FIPSStandCtyCode', 'EntityofFile',
       'SecondaryEntityOfFile', 'DateofFile', 'DateofCreation', 'FileLength',
       'StateName', 'StateNameAbbreviation', 'CountyName',
       ...
       'condition_Drug Abuse', 'condition_HIV/AIDS', 'condition_Heart Failure',
       'condition_Hepatitis', 'condition_Hyperlipidemia',
       'condition_Hypertension', 'condition_Ischemic Heart Disease',
       'condition_Osteoporosis', 'condition_Psychotic Disorders',
       'condition_Stroke'],
      dtype='object', length=7232)

In [14]:
df.shape

(1002, 7232)